## `A <-> B` reaction, with 1st-order kinetics in both directions,
### taken to equilibrium

Diffusion NOT done

### TAGS :  "reactions 2D"

In [1]:
LAST_REVISED = "Dec. 16, 2024"
LIFE123_VERSION = "1.0-rc.1"        # Library version this experiment is based on

In [2]:
#import set_path                    # Using MyBinder?  Uncomment this before running the next cell!

In [3]:
#import sys
#sys.path.append("C:/some_path/my_env_or_install")   # CHANGE to the folder containing your venv or libraries installation!
# NOTE: If any of the imports below can't find a module, uncomment the lines above, or try:  import set_path   

from experiments.get_notebook_info import get_notebook_basename

from life123 import UniformCompartment, BioSim2D, GraphicLog

In [4]:
# Initialize the HTML logging
log_file = get_notebook_basename() + ".log.htm"    # Use the notebook base filename for the log file
# Set up the use of some specified graphic (Vue) components
GraphicLog.config(filename=log_file,
                  components=["vue_cytoscape_2"],
                  extra_js="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.21.2/cytoscape.umd.js")

-> Output will be LOGGED into the file 'reaction_1.log.htm'


In [5]:
# Initialize the system.  NOTE: Diffusion not done
uc = UniformCompartment(names=["A", "B"])

# Reaction A <-> B , with 1st-order kinetics in both directions
uc.add_reaction(reactants="A", products="B", forward_rate=3., reverse_rate=2.)

uc.describe_reactions()

Number of reactions: 1 (at temp. 25 C)
0: A <-> B  (kF = 3 / kR = 2 / delta_G = -1,005.1 / K = 1.5) | 1st order in all reactants & products
Set of chemicals involved in the above reactions: {'B', 'A'}


In [6]:
# Send the plot to the HTML log file
uc.plot_reaction_network("vue_cytoscape_2")

[GRAPHIC ELEMENT SENT TO LOG FILE `reaction_1.log.htm`]


In [7]:
bio = BioSim2D(n_bins=(3,4), reaction_handler=uc)

bio.set_bin_conc_all_species(bin_x=0, bin_y=0, conc_list=[10.,50.])
bio.set_bin_conc_all_species(bin_x=0, bin_y=1, conc_list=[20.,35.])
bio.set_bin_conc_all_species(bin_x=2, bin_y=3, conc_list=[5.,100.])

bio.describe_state()

SYSTEM STATE at Time t = 0:
Species `A`:
      0     1    2    3
0  10.0  20.0  0.0  0.0
1   0.0   0.0  0.0  0.0
2   0.0   0.0  0.0  5.0
Species `B`:
      0     1    2      3
0  50.0  35.0  0.0    0.0
1   0.0   0.0  0.0    0.0
2   0.0   0.0  0.0  100.0


## First step

In [8]:
# First step (NOTE: here we're using a lower-level function that doesn't update the system state;
#                   it only computes the delta_reactions array)
bio.reaction_step(delta_time=0.1)
print("bio.delta_reactions:\n", bio.delta_reactions)

bio.delta_reactions:
 [[[  7.    1.    0.    0. ]
  [  0.    0.    0.    0. ]
  [  0.    0.    0.   18.5]]

 [[ -7.   -1.    0.    0. ]
  [  0.    0.    0.    0. ]
  [  0.    0.    0.  -18.5]]]


In [9]:
bio.system += bio.delta_reactions       # Matrix operation to update all the concentrations
bio.system_time += 0.1

bio.describe_state()

SYSTEM STATE at Time t = 0.1:
Species `A`:
      0     1    2     3
0  17.0  21.0  0.0   0.0
1   0.0   0.0  0.0   0.0
2   0.0   0.0  0.0  23.5
Species `B`:
      0     1    2     3
0  43.0  34.0  0.0   0.0
1   0.0   0.0  0.0   0.0
2   0.0   0.0  0.0  81.5


## Second step

In [10]:
# NOTE: now, we're using a highel-level function that also updates the system state
bio.react(time_step=0.1, n_steps=1)
bio.describe_state()

SYSTEM STATE at Time t = 0.2:
Species `A`:
      0     1    2      3
0  20.5  21.5  0.0   0.00
1   0.0   0.0  0.0   0.00
2   0.0   0.0  0.0  32.75
Species `B`:
      0     1    2      3
0  39.5  33.5  0.0   0.00
1   0.0   0.0  0.0   0.00
2   0.0   0.0  0.0  72.25


## Many more steps, to equilibrium

In [11]:
bio.react(time_step=0.1, n_steps=8)
bio.describe_state()

SYSTEM STATE at Time t = 0.9999999999999999:
Species `A`:
           0          1    2          3
0  23.986328  21.998047  0.0   0.000000
1   0.000000   0.000000  0.0   0.000000
2   0.000000   0.000000  0.0  41.963867
Species `B`:
           0          1    2          3
0  36.013672  33.001953  0.0   0.000000
1   0.000000   0.000000  0.0   0.000000
2   0.000000   0.000000  0.0  63.036133


In [12]:
bio.react(time_step=0.1, n_steps=10)
bio.describe_state()

SYSTEM STATE at Time t = 2.0000000000000004:
Species `A`:
           0          1    2          3
0  23.999987  21.999998  0.0   0.000000
1   0.000000   0.000000  0.0   0.000000
2   0.000000   0.000000  0.0  41.999965
Species `B`:
           0          1    2          3
0  36.000013  33.000002  0.0   0.000000
1   0.000000   0.000000  0.0   0.000000
2   0.000000   0.000000  0.0  63.000035


### The system has now reached equilibrium
### in individual bins, which remain separate because we're NOT doing diffusion in this experiment

Verify the equilibrium in each of the active bins

In [13]:
bio.reaction_dynamics.is_in_equilibrium(rxn_index=0, conc={"A": 23.99998665, "B": 36.00001335})

A <-> B
Final concentrations: [A] = 24 ; [B] = 36
1. Ratio of reactant/product concentrations, adjusted for reaction orders: 1.5
    Formula used:  [B] / [A]
2. Ratio of forward/reverse reaction rates: 1.5
Discrepancy between the two values: 9.271e-05 %
Reaction IS in equilibrium (within 1% tolerance)



True

In [14]:
bio.reaction_dynamics.is_in_equilibrium(rxn_index=0, conc={"A": 21.99999809, "B": 33.00000191})

A <-> B
Final concentrations: [A] = 22 ; [B] = 33
1. Ratio of reactant/product concentrations, adjusted for reaction orders: 1.5
    Formula used:  [B] / [A]
2. Ratio of forward/reverse reaction rates: 1.5
Discrepancy between the two values: 1.447e-05 %
Reaction IS in equilibrium (within 1% tolerance)



True

In [15]:
bio.reaction_dynamics.is_in_equilibrium(rxn_index=0, conc={"A": 41.99996471, "B": 63.00003529}, explain=False)

True